<a href="https://colab.research.google.com/github/CiaraFarrellSETU/Dissertation_code/blob/main/inception_v3_6classes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from timeit import default_timer as timer

start = timer()

In [ ]:
# %tensorflow_version 2.x
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns

import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras import backend, models, layers, optimizers, regularizers, metrics
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.utils import plot_model


from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

from PIL import Image

import shutil

np.random.seed(0)
tf.random.set_seed(0)

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import zipfile

with zipfile.ZipFile('/content/gdrive/MyDrive/dogeye.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/unzipp_d')

In [ ]:
base_dir = '/content/unzipp_d/dogeye'

train_dir = '/content/unzipp_d/dogeye/train'
cat_train_dir = '/content/unzipp_d/dogeye/train/cataract'
healthy_train_dir = '/content/unzipp_d/dogeye/train/healthy'
mas_train_dir = '/content/unzipp_d/dogeye/train/mas'
conj_train_dir = '/content/unzipp_d/dogeye/train/conj'
ulcer_train_dir = '/content/unzipp_d/dogeye/train/ulcer'
nuc_train_dir = '/content/unzipp_d/dogeye/train/nuc'

test_dir = '/content/unzipp_d/dogeye/test'
cat_test_dir = '/content/unzipp_d/dogeye/test/cataract'
healthy_test_dir = '/content/unzipp_d/dogeye/test/healthy'
mas_test_dir = '/content/unzipp_d/dogeye/test/mas'
conj_test_dir = '/content/unzipp_d/dogeye/test/conj'
ulcer_test_dir = '/content/unzipp_d/dogeye/test/ulcer'
nuc_test_dir = '/content/unzipp_d/dogeye/test/nuc'

val_dir = '/content/unzipp_d/dogeye/valid'
cat_val_dir = '/content/unzipp_d/dogeye/valid/cataract'
healthy_val_dir = '/content/unzipp_d/dogeye/valid/healthy'
mas_val_dir = '/content/unzipp_d/dogeye/valid/mas'
conj_val_dir = '/content/unzipp_d/dogeye/valid/conj'
ulcer_val_dir = '/content/unzipp_d/dogeye/valid/ucler'
nuc_val_dir = '/content/unzipp_d/dogeye/valid/nuc'

In [ ]:
import os
#train
num_cat_train = len(os.listdir(cat_train_dir))
num_healthy_train = len(os.listdir(healthy_train_dir))

num_mas_train = len(os.listdir(mas_train_dir))
num_conj_train = len(os.listdir(conj_train_dir))
num_ulcer_train = len(os.listdir(ulcer_train_dir))
num_nuc_train = len(os.listdir(nuc_train_dir))

#test
num_cat_test = len(os.listdir(cat_test_dir))
num_healthy_test = len(os.listdir(healthy_test_dir))

num_mas_test = len(os.listdir(mas_test_dir))
num_conj_test = len(os.listdir(conj_test_dir))
num_ulcer_test = len(os.listdir(ulcer_test_dir))
num_nuc_test = len(os.listdir(nuc_test_dir))

#val
num_cat_val = len(os.listdir(cat_val_dir))
num_healthy_val = len(os.listdir(healthy_val_dir))

num_mas_val = len(os.listdir(mas_val_dir))
num_conj_val = len(os.listdir(conj_val_dir))
num_ulcer_val = len(os.listdir(ulcer_val_dir))
num_nuc_val = len(os.listdir(nuc_val_dir))

In [ ]:
print("--")
print("Total Training cat Images",num_cat_train)
print("Total Training conj Images",num_conj_train)

print("Total Training healthy Images",num_healthy_train)
print("Total Training mas Images",num_mas_train)
print("Total Training nuc Images",num_nuc_train)
print("Total Training ulcer Images",num_ulcer_train)
print("--")
print("Total test cat Images",num_cat_test)
print("Total test healthy Images",num_healthy_test)

print("Total test mas Images",num_mas_test)
print("Total Training conj Images",num_conj_test)
print("Total Training ulcer Images",num_ulcer_test)
print("Total Training nuc Images",num_nuc_test)
print("--")
print("Total validation cat Images",num_cat_val)
print("Total validation healthy Images",num_healthy_val)

print("Total validation mas Images",num_mas_val)
print("Total Training conj Images",num_conj_val)
print("Total Training ulcer Images",num_ulcer_val)
print("Total Training nuc Images",num_nuc_val)
print("--")

In [ ]:
total_train = num_cat_train+num_mas_train+num_conj_train+num_ulcer_train+num_nuc_train+num_healthy_train
total_validation = num_cat_val+num_mas_val+num_conj_val+num_ulcer_val+num_nuc_val+num_healthy_val
total_test = num_cat_test+num_mas_test+num_conj_test+num_ulcer_test+num_nuc_test+num_healthy_test

total_images = total_train + total_validation + total_test

print("--")
print("Total Training Images",total_train)
print("--")
print("Total Validation Images",total_validation)
print("--")
print("Total Testing Images",total_test)
print("--")
print("Total Images",total_images)
print("--")

In [ ]:
img_size = [250, 250]
epochs = 50

In [ ]:
base_dir = '/content/unzipp_d/dogeye'

training = (base_dir + '/train')
test = (base_dir + '/test')

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

test_datagen = ImageDataGenerator(
    rescale=1./255
)

train_generator = train_datagen.flow_from_directory(
    training,
    target_size=(250, 250),
    batch_size=32,
    class_mode='categorical',
    subset='training',
    color_mode='rgb'
)

validation_generator = train_datagen.flow_from_directory(
    training,
    target_size=(250, 250),
    batch_size=32,
    class_mode='categorical',
    subset='validation',
    color_mode='rgb'
)

test_generator = train_datagen.flow_from_directory(
    test,
    target_size=(250, 250),
    batch_size=32,
    class_mode='categorical',
    color_mode='rgb'
)

class_names = ['cat', 'conj','healthy', 'mas','nuc','ulcer'] # put in order
train_generator.class_names = class_names
validation_generator.class_names = class_names
test_generator.class_names = class_names

steps_per_epoch = train_generator.n // train_generator.batch_size
print('steps per epoch: ',steps_per_epoch)

In [ ]:
# for images, labels in train_generator:
#   plt.figure(figsize=(10,10))
#   for i in range(9):
#     plt.subplot(330 + 1 + i)
#     plt.imshow(images[i])
#     class_name = train_generator.class_names[np.where(labels[i] == 1)[0][0]]
#     plt.title(class_name)
#     plt.axis('off')
#   plt.show()
#   break

In [ ]:
backend.clear_session()
inception_base = InceptionV3(
    weights = 'imagenet',
    include_top = False,
    input_shape = (*img_size, 3)
    )

inception_base.trainable = False

visible = layers.Input(shape=(250,250,3), name='input')
x = inception_base(visible)

x = layers.Flatten()(x)
x = layers.Dense(512, activation = 'relu')(x)

output = layers.Dense(6, activation='softmax', name='output')(x)

model = Model(inputs=visible, outputs=output)

model.compile(optimizer = 'adam',
               loss = 'categorical_crossentropy',
               metrics = [metrics.AUC(name='auc')]
             )

print(model.summary())

history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=epochs,
    validation_data=validation_generator,
    verbose = 1,
    callbacks=[
               EarlyStopping(
                   monitor='val_auc',
                   patience=15,
                   restore_best_weights = True
                   )
               ]
            )


In [ ]:

test_loss, test_accuracy = model.evaluate(test_generator)
print("Test accuracy:", test_accuracy)
print("Test loss:", test_loss)


In [ ]:
import numpy as np
y_true = test_generator.classes
y_pred_prob = model.predict(test_generator)
y_pred = np.argmax(y_pred_prob, axis=1)

report = classification_report(y_true, y_pred, target_names=class_names)
print(report)


In [ ]:
import matplotlib.pyplot as plt
cm = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:\n")
print(cm)

In [ ]:
end = timer()
print(end - start)